In [9]:
import numpy as np
import pandas as pd
import geopandas as gpd
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from scipy.spatial.distance import pdist, squareform
from sklearn.neighbors import BallTree
import torch_geometric
from torch_geometric.nn import GATConv
from torch_geometric.nn.norm import BatchNorm
from torch.utils.data import random_split
import libpysal
from libpysal.weights import Queen

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [11]:
data = gpd.read_file("/data/election.geojson")
# Lambert Conformal Conic
data = data.to_crs("+proj=lcc +lat_1=33 +lat_2=45 +lat_0=39 +lon_0=-96 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs")

y = data['pct_gop_16'].values

X_vars = ['AGE135214','AGE775214','POP815213',
          'SEX255214','RHI125214','POP715213','EDU635213','EDU685213',
          'LFE305213','HSG445213']
X = data[X_vars].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

features = torch.FloatTensor(X_scaled).to(device)
labels = torch.FloatTensor(y).to(device)

In [166]:
def calculate_weighted_similarity(X, sigma_values, weights):
    u = X[:, np.newaxis, :]
    v = X[np.newaxis, :, :]
    sq_diff = (u - v) ** 2

    E_i = np.exp(-((sq_diff) / (2 * (sigma_values))))

    weighted_similarity = np.average(E_i, axis=2, weights=weights)
    return weighted_similarity

sigma_values = np.var(X, axis=0)
weights = np.array([0.393, 0.3, 1.02, 0.511, 1.921, 0.276, 0.5388, 2.356, 0.589, 1.06])
# weights = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

base_similarity_matrix = calculate_weighted_similarity(X, sigma_values, weights)

queen_w = Queen.from_dataframe(data)

# Compute neighbor mean vectors based on Queen rule
def compute_neighbor_mean_vectors(X, neighbors_dict):
    N, D = X.shape
    neighbor_means = np.zeros((N, D))
    for i in range(N):
        nbrs = neighbors_dict[i]
        if len(nbrs) == 0:
            neighbor_means[i] = X[i]
        else:
            neighbor_means[i] = np.mean(X[nbrs], axis=0)
    return neighbor_means

neighbor_means = compute_neighbor_mean_vectors(X_scaled, queen_w.neighbors)

# Calculae the neighorhood similarity
def calculate_neighbor_similarity(neighbor_means, sigma_values, weights):
    u = neighbor_means[:, np.newaxis, :]
    v = neighbor_means[np.newaxis, :, :]

    sq_diff = (u - v) ** 2

    E_i = np.exp(-((sq_diff) / (2 * sigma_values)))
    weighted_neighbor_sim = np.average(E_i, axis=2, weights=weights)
    return weighted_neighbor_sim

neighbor_similarity_matrix = calculate_neighbor_similarity(neighbor_means, sigma_values, weights)


<ipython-input-166-f633dfe526c9>:17: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  queen_w = Queen.from_dataframe(data)


In [167]:
# S_l +S_n: α * base_similarity + β * neighbor_similarity

alpha = 1.0
beta = 1.0
combined_similarity_matrix = alpha * base_similarity_matrix + beta * neighbor_similarity_matrix


dist_matrix = squareform(pdist(combined_similarity_matrix, metric='euclidean'))
nearest_neighbors = np.argsort(dist_matrix, axis=1)[:, 1:31]

adj = np.zeros((X.shape[0], X.shape[0]))
for i in range(nearest_neighbors.shape[0]):
    for j in nearest_neighbors[i]:
        adj[i, j] = combined_similarity_matrix[i, j]
adj = (adj + adj.T) / 2

adj = torch.FloatTensor(adj).to(device)

In [168]:
dataset = TensorDataset(features, labels)
train_size = int(0.1 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

all_indices = np.arange(len(dataset))
np.random.seed(42)
np.random.shuffle(all_indices)

train_indices = all_indices[:train_size]
val_indices = all_indices[train_size:train_size + val_size]
test_indices = all_indices[train_size + val_size:]

In [169]:
class GAT(nn.Module):
  def __init__(self, f_in, n_classes, hidden=[128], heads=18, dropout=0.2):
    super(GAT, self).__init__()
    self.conv1 = GATConv(f_in, hidden[0], heads=heads, dropout=dropout, edge_dim=1)
    self.bn1 = BatchNorm(hidden[0] * heads)
    self.conv2 = GATConv(hidden[0] * heads, n_classes, heads=heads, concat=False, dropout=dropout, edge_dim=1)
    self.bn2 = BatchNorm(n_classes)
    self.dropout = dropout

  def forward(self, x, edge_index, edge_attr):
    x = F.dropout(x, p=self.dropout, training=self.training)
    x = F.elu(self.conv1(x, edge_index, edge_attr))
    x = self.bn1(x)
    x = F.dropout(x, p=self.dropout, training=self.training)
    x = self.conv2(x, edge_index, edge_attr)
    x = self.bn2(x)
    return x


edge_index = torch_geometric.utils.dense_to_sparse(adj)[0].to(device)
edge_attr = adj[edge_index[0], edge_index[1]].to(device)

def filter_edge_index(edge_index, edge_attr, indices):
    mask = torch.isin(edge_index, torch.tensor(indices).to(device)).all(dim=0)
    filtered_edge_index = edge_index[:, mask]
    filtered_edge_attr = edge_attr[mask]
    remap = {old_idx: new_idx for new_idx, old_idx in enumerate(indices)}
    remapped_edge_index = torch.stack([
        torch.tensor([remap[int(i)] for i in filtered_edge_index[0]]).to(device),
        torch.tensor([remap[int(i)] for i in filtered_edge_index[1]]).to(device)
    ])
    return remapped_edge_index, filtered_edge_attr

In [173]:
def train_model(model, features, edge_index, edge_attr, labels, train_indices, val_indices, epochs=2000, lr=1e-3):
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=1e-5)
    model.train()
    best_val_loss = float('inf')
    early_stopping_patience = 20
    train_edge_index, train_edge_attr = filter_edge_index(edge_index, edge_attr, train_indices)
    val_edge_index, val_edge_attr = filter_edge_index(edge_index, edge_attr, val_indices)
    for epoch in range(epochs):
        optimizer.zero_grad()
        output = model(features[train_indices], train_edge_index, train_edge_attr)
        loss = F.mse_loss(output, labels[train_indices].unsqueeze(1))
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        model.eval()
        with torch.no_grad():
            val_output = model(features[val_indices], val_edge_index, val_edge_attr)
            val_loss = F.mse_loss(val_output, labels[val_indices].unsqueeze(1))
        model.train()

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            patience = 0
        else:
            patience += 1

        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Training Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}')

        if patience >= early_stopping_patience:
            print('Early stopping.')
            break

    model.load_state_dict(best_model_state)
    return model


In [174]:
model = GAT(f_in=features.shape[1], n_classes=1, hidden=[40], heads=20, dropout=0.2).to(device)
trained_model = train_model(model, features, edge_index, edge_attr, labels, train_indices, val_indices, epochs=2000, lr=1e-2)

Epoch 10/2000, Training Loss: 0.9250, Validation Loss: 0.2579
Epoch 20/2000, Training Loss: 0.6790, Validation Loss: 0.2216
Epoch 30/2000, Training Loss: 0.4893, Validation Loss: 0.2067
Epoch 40/2000, Training Loss: 0.3416, Validation Loss: 0.1615
Epoch 50/2000, Training Loss: 0.2332, Validation Loss: 0.1252
Epoch 60/2000, Training Loss: 0.1601, Validation Loss: 0.0830
Epoch 70/2000, Training Loss: 0.1035, Validation Loss: 0.0552
Epoch 80/2000, Training Loss: 0.0669, Validation Loss: 0.0366
Epoch 90/2000, Training Loss: 0.0456, Validation Loss: 0.0266
Epoch 100/2000, Training Loss: 0.0324, Validation Loss: 0.0199
Epoch 110/2000, Training Loss: 0.0219, Validation Loss: 0.0153
Epoch 120/2000, Training Loss: 0.0164, Validation Loss: 0.0128
Epoch 130/2000, Training Loss: 0.0138, Validation Loss: 0.0114
Epoch 140/2000, Training Loss: 0.0108, Validation Loss: 0.0108
Epoch 150/2000, Training Loss: 0.0094, Validation Loss: 0.0104
Epoch 160/2000, Training Loss: 0.0091, Validation Loss: 0.0102
E

In [177]:
def evaluate_model(model, features, edge_index, edge_attr, labels, indices):
    model.eval()
    test_edge_index, test_edge_attr = filter_edge_index(edge_index, edge_attr, indices)
    with torch.no_grad():
        predictions = model(features[indices], test_edge_index, test_edge_attr)
        if isinstance(predictions, tuple):
            predictions = predictions[0]
        predictions = predictions.squeeze()
        mse = F.mse_loss(predictions, labels[indices].squeeze())
        mae = F.l1_loss(predictions, labels[indices].squeeze())
        y_true = labels[indices].cpu().numpy()
        y_pred = predictions.cpu().numpy()
    return predictions, mse, mae

predictions, mse, mae = evaluate_model(trained_model, features, edge_index, edge_attr, labels, test_indices)
print(f'Test MSE: {mse.item():.4f}')
print(f'Test MAE: {mae.item():.4f}')


Test MSE: 0.0105
Test MAE: 0.0789
